In [13]:
import csv
import pytz
from datetime import datetime
import json
import numpy
from scipy import stats

#convert UTC time to IST
def convert_to_ist(utc_time):
    ist = pytz.timezone('Asia/Kolkata')
    return utc_time.astimezone(ist).strftime('%Y-%m-%d %H:%M:%S')

def flatten_json(json_data):
    flat_data = []
    for item in json_data:
        flat_item = {}
        track_details = item.get('trackDetails', [])
        for track_detail in  track_details:

            flat_item['Tracking number'] = track_detail.get('trackingNumber')
            dates_or_times = track_detail.get('datesOrTimes', [])   #pickup and delivery date/time
            flat_item['Pickup Date Time'] = get_date_time(dates_or_times, 'ACTUAL_PICKUP')
            flat_item['Delivery Date Time'] = get_date_time(dates_or_times, 'ACTUAL_DELIVERY')
            flat_item['Days taken for delivery'] = calculate_days(flat_item['Pickup Date Time'], flat_item['Delivery Date Time'])
            flat_item['Shipment weight'] = track_detail.get('shipmentWeight', {}).get('value')

            events = track_detail.get('events', [])     #pickup and delivery location details
            flat_item['Number of delivery attempts'] = calculate_delivery_attempts(events)
            pickup_location = next((get_pickup_location_details(event) for event in events if event.get('eventDescription') == 'Picked up'), None)
            flat_item['Pickup Location'] = pickup_location
            delivery_location = next((get_delivery_location_details(event) for event in events if event.get('eventDescription') == 'Delivered'),None)
            flat_item['Delivery Location'] = delivery_location

            payment_method =get_payment_method(track_detail)
            flat_item['Payment type'] = payment_method

        flat_data.append(flat_item)
    return flat_data

# return the payment method as C.O.D or prepaid (Assume it as prepaid if there is no mention of C.O.D)
def get_payment_method(track_detail):
    # check if payment method is C.O.D. based on statusDetail
    cod_payment = any(detail['reason'] == '11' for detail in track_detail['statusDetail']['ancillaryDetails'])
    if not cod_payment:     #check if payment method is C.O.D. based on events
        events = track_detail.get('events', [])
        cod_payment = any(event.get('statusExceptionCode') == '11' for event in events)
    #else consider it as prepaid
    payment_method = 'C.O.D' if cod_payment else 'Prepaid'
    return payment_method

# to get pick up location city, pincode, state code
def get_pickup_location_details(event):
    address = event.get('address', {})
    city = address.get('city')
    postal_code = address.get('postalCode')
    state = address.get('stateOrProvinceCode')

    return city, postal_code, state

# returns delivery location city, state code, pincode
def get_delivery_location_details(event):
    address = event.get('address', {})
    city = address.get('city')
    postal_code = address.get('postalCode')
    state = address.get('stateOrProvinceCode')

    return city, postal_code, state

# to get the date/time from the given type
def get_date_time(dates_or_times, type):
    for date_time in dates_or_times:
        if date_time['type'] == type:
            return convert_to_ist(datetime.fromisoformat(date_time['dateOrTimestamp']))     #converts UTC time to IST
    return None

#calculate number of days between two dates
def calculate_days(start_date, end_date):
    if start_date and end_date:
        start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S') #converts the input dates to datetime objects 
        end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
        return (end_date - start_date).days     #returns the no. of days
    return None

# calculate number of delivery attempts
def calculate_delivery_attempts(events):
    delivery_attempts = 0
    for event in events:
        #check how many times delivery attempt is made
        if (event.get('arrivalLocation') == 'DELIVERY_LOCATION'):
            delivery_attempts += 1 
    return delivery_attempts


# write data to CSV file
def write_to_csv(data, filename):
    headers = ['Tracking number', 'Payment type', 'Pickup Date Time', 'Delivery Date Time', 'Days taken for delivery',
               'Shipment weight', 'Pickup Location','Delivery Location', 'Number of delivery attempts']
    with open(filename, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)

#to caluculate mean , median , mode of no.of days taken for delivery and no. of delivery attempts
def calculate_MMM(flat_data):
    Total_days = []
    Total_attempts = []
    for data in flat_data:
        days = data.get('Days taken for delivery')
        Total_days.append(days)
        attempts = data.get('Number of delivery attempts')
        Total_attempts.append(attempts)
    Total_attempts = numpy.array(Total_attempts)
    Total_days = numpy.array(Total_days)
    print(f'Days taken for delivery: \n mean = {numpy.mean(Total_days)} \n median = {numpy.median(Total_days)} \n mode = {stats.mode(Total_days).mode}')
    print(f'Number of delivery attempts: \n mean = {numpy.mean(Total_attempts)} \n median = {numpy.median(Total_attempts)} \n mode = {stats.mode(Total_attempts).mode}')

#input JSON file path
json_file_path = 'DE_data_basic.json'

#read JSON file
with open(json_file_path, 'r') as json_file:
    json_data = json.load(json_file)

flattened_data = flatten_json(json_data)
calculate_MMM(flattened_data)
write_to_csv(flattened_data, './shipment.csv')


Days taken for delivery: 
 mean = 3.2626262626262625 
 median = 3.0 
 mode = 3
Number of delivery attempts: 
 mean = 1.292929292929293 
 median = 1.0 
 mode = 1
